In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds

In [2]:
# tfds.load returns a tf.data.Dataset object and a tf.core.DatasetInfo object. The latter contains info about how the set is split 
# into test and train, number of examples and names of features
dataset, metadata = tfds.load('cifar10', as_supervised=True, with_info=True) 
train_dataset, test_dataset = dataset['train'], dataset['test']

In [8]:
# Normalise data
def minmaxScaling(images, labels):
    images = tf.cast(images, tf.float32)
    images = images/255
    return images, labels

train_dataset = train_dataset.map(minmaxScaling)
test_dataset = test_dataset.map(minmaxScaling)

# Size of training and test sets
train_size = metadata.splits['train'].num_examples
test_size = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(train_size))
print("Number of test examples:     {}".format(test_size))

Number of training examples: 50000
Number of test examples:     10000


In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, 3, padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=2, padding='same'),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation=('relu')),
    tf.keras.layers.MaxPooling2D(pool_size=2, padding='same'),
    tf.keras.layers.Conv2D(32, 3, padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=2, padding='same'),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation=('relu')),
    tf.keras.layers.MaxPooling2D(pool_size=2, padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=10, activation=('softmax'))
])
# Compile model
model.compile(loss='sparse_categorical_crossentropy', 
              #optimizer = 'adam',
              optimizer = tf.keras.optimizers.Adam(0.1), 
              metrics=['accuracy'])